In [1]:
from IPython.display import Image

import sys
sys.path.append("/home/xilinx/fos/")

import udmalib.udma as udma
import cv2, numpy as np
import proto.fpga_rpc_c as fpga_rpc_client

In [4]:
inputfile = "/home/xilinx/fos/clients/simple_cpp/eiffel.jpg"

In [ ]:
# bit hacking functions for driving unit
int32max = (1 << 32) - 1
def top32(data): return int((data >> 32) & int32max)
def bot32(data): return int(data & int32max)
 
# allocate a buffer
ubuffers = udma.udmas()
fpga_rpc = fpga_rpc_client.Client()
bufno = fpga_rpc.Alloc()
if (bufno < 0): raise "could not allocate buffers"
ubuffer = ubuffers.getDevice(bufno)
buffer = ubuffer.map()
 
# read in images
input = cv2.imread(inputfile, cv2.IMREAD_GRAYSCALE)
height = input.shape[0]
width = input.shape[1]
size = width*height
 
# calculate pointers
buf0addr = ubuffer.phys_addr
buf1addr = ubuffer.phys_addr + size
 
for y in range(height):
    for x in range(width):
        buffer[y*width+x] = input[y, x]

# run hardware unit
fpga_rpc.Run([{
    "name": "Partial_sobel",
    "params": {
        "in_pixels":      bot32(buf0addr),
        "in_pixels_msb":  top32(buf0addr),
        "out_pixels":     bot32(buf1addr),
        "out_pixels_msb": top32(buf1addr),
        "im_width":       width,
        "im_height":      height
    }
}]*10)
 
# write output to file
output = np.zeros(input.shape, dtype=input.dtype)
for y in range(height):
    for x in range(width):
        output[y, x] = buffer[size + y*width+x]
cv2.imwrite("sobel_out.jpg", output)

fpga_rpc.Free(bufno)

Image("sobel_out.jpg")
